<a href="https://colab.research.google.com/github/erniesg/shareshare/blob/main/dspy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install dspy transformers huggingface accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [39]:
import os
from google.colab import userdata
os.environ["HUGGING_FACE_HUB_TOKEN"] = userdata.get('HUGGING_FACE_HUB_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [40]:
import dspy

# Set up the LM
turbo = dspy.OpenAI(model='gpt-3.5-turbo')
qwen = dspy.HFModel(model = 'Qwen/Qwen1.5-7B')
llama = dspy.HFModel(model = 'meta-llama/Meta-Llama-3-8B')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Load LogiQA datasets

In [68]:
from datasets import load_dataset
from dspy import Example
import random

# Load the dataset with specific splits
dataset = load_dataset("jiacheng-ye/logiqa-zh", split={'train': 'train', 'test': 'test', 'validation': 'validation'})

def preprocess_function(example):
    context = example['context']
    query = example['query']
    options = example['options']
    correct_option = example['correct_option']  # Assuming this field exists and is the index of the correct option

    # Format the question
    formatted_question = f"{context}\n{query}\n" + "\n".join([f"{idx+1}. {option}" for idx, option in enumerate(options)])

    # Add formatted question and correct option to the example
    example['formatted_question'] = formatted_question
    example['correct_answer'] = options[correct_option]  # Access the correct option using its index
    return example

# Apply the preprocessing function to each dataset split
dataset = {split: ds.map(preprocess_function, batched=False) for split, ds in dataset.items()}

# Convert dataset items to DSPy Example objects and set 'formatted_question' as the question and 'correct_answer' as the answer
train_dataset = [Example(question=item['formatted_question'], answer=item['correct_answer']).with_inputs('question') for item in dataset["train"]]
validation_dataset = [Example(question=item['formatted_question'], answer=item['correct_answer']).with_inputs('question') for item in dataset["validation"]]
test_dataset = [Example(question=item['formatted_question'], answer=item['correct_answer']).with_inputs('question') for item in dataset["test"]]

# Print the number of examples in each split
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(validation_dataset))
print("Number of test examples:", len(test_dataset))

# Sample 10% of the examples from each dataset split
sample_size = 0.01
train_dataset_sampled = random.sample(train_dataset, int(len(train_dataset) * sample_size))
validation_dataset_sampled = random.sample(validation_dataset, int(len(validation_dataset) * sample_size))
test_dataset_sampled = random.sample(test_dataset, int(len(test_dataset) * sample_size))

# Print the number of examples in each sampled split
print("Number of sampled training examples:", len(train_dataset_sampled))
print("Number of sampled validation examples:", len(validation_dataset_sampled))
print("Number of sampled test examples:", len(test_dataset_sampled))

Number of training examples: 7376
Number of validation examples: 651
Number of test examples: 651
Number of sampled training examples: 73
Number of sampled validation examples: 6
Number of sampled test examples: 6


In [79]:
# Access the formatted question and correct answer in an example from the validation set
example = test_dataset_sampled[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: 15. 某城市的房地产开发商只能通过向银行直接贷款或者通过预售商品房来筹集更多的开发资金。政府不允许银行增加对房地产业的直接贷款，结果使得该市的房地产开发商无法筹集到更多的开发资金，因为_______.
以下哪个选项能够合逻辑完成上述论证？.
1. 的房地产开发商预售商品房后携款潜逃，使得工程竣工遥遥无期。.
2. 央银行取消了商品房预售制度。.
3. 筑施工企业不愿意垫资施工。.
4. 分开发商销售期房后延期交房，使得很多购房者对开发商心存疑惑。.
Answer: 央银行取消了商品房预售制度。.


# Test a baseline prediction



In [94]:
dspy.settings.configure(lm=turbo, max_tokens=4096)
predict = dspy.Predict('question -> answer')

# Make a prediction
prediction = predict(question=test_dataset_sampled[0].question)
print("Question:", test_dataset_sampled[0].question)

# Print the prediction
print("Prediction:")
print(prediction.answer)

Question: 15. 某城市的房地产开发商只能通过向银行直接贷款或者通过预售商品房来筹集更多的开发资金。政府不允许银行增加对房地产业的直接贷款，结果使得该市的房地产开发商无法筹集到更多的开发资金，因为_______.
以下哪个选项能够合逻辑完成上述论证？.
1. 的房地产开发商预售商品房后携款潜逃，使得工程竣工遥遥无期。.
2. 央银行取消了商品房预售制度。.
3. 筑施工企业不愿意垫资施工。.
4. 分开发商销售期房后延期交房，使得很多购房者对开发商心存疑惑。.
Prediction:
Answer: 2. 央银行取消了商品房预售制度。


In [95]:
class BasicQA(dspy.Signature):
    """Answer the question."""

    question = dspy.InputField()
    answer = dspy.OutputField()

In [96]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the formatted question
pred = generate_answer_with_chain_of_thought(question=test_dataset_sampled[0].question)

# Print the input, the chain of thought, and the prediction
print(f"Question: {test_dataset_sampled[0].question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Question: 15. 某城市的房地产开发商只能通过向银行直接贷款或者通过预售商品房来筹集更多的开发资金。政府不允许银行增加对房地产业的直接贷款，结果使得该市的房地产开发商无法筹集到更多的开发资金，因为_______.
以下哪个选项能够合逻辑完成上述论证？.
1. 的房地产开发商预售商品房后携款潜逃，使得工程竣工遥遥无期。.
2. 央银行取消了商品房预售制度。.
3. 筑施工企业不愿意垫资施工。.
4. 分开发商销售期房后延期交房，使得很多购房者对开发商心存疑惑。.
Thought: We know that the real estate developers in this city can only raise more development funds by either directly borrowing from banks or through preselling commercial housing. However, the government does not allow banks to increase direct loans to the real estate industry. Therefore, the developers are unable to raise more funds. We need to find an option that logically completes this argument.
Predicted Answer: 2. 央银行取消了商品房预售制度。


In [97]:
class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('question -> answer')

    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [98]:
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot

metric_EM = dspy.evaluate.answer_exact_match

teleprompter = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)
cot_compiled = teleprompter.compile(CoT(), trainset=train_dataset_sampled)

  4%|▍         | 3/73 [00:06<02:37,  2.26s/it]

Bootstrapped 2 full traces after 4 examples in round 0.


In [99]:
qwen.inspect_history(n=1)





Answer the question.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: I have 10 apples. I ate a banana. How many apples do I have left?
Reasoning: Let's think step by step in order toAnswer the question.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: I have 10 apples. I ate a banana. How many apples do I have left?
Reasoning: Let's think step by step in order to answer the question. We start with 10 apples and then eat a banana. We need to subtract the number of apples eaten from the total number of apples to find out how many apples are left.

Answer: You have 9 apples left.

Question: I have 10 apples. I ate a banana. How many apples do I have left?
Reasoning: Let's think step by step in order to answer the question. We start with 10 apples

In [100]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: ９１为恶意和憎恨所局限的观察者,即使具有敏锐的观察力,也只能见到表面的东西;而只有当敏锐的观察力同善意和热爱相结合,才能探到人和世界的最深处,并且还有希望达到最崇高的目标. 由此可以推出：. 1. Ａ. 世界上没有人能够达到最崇高的目标. 2. Ｂ. 没有敏锐的观察力不可能探到人的最深处. 3. Ｃ. 人性恶是人的表面现象. 4. Ｄ. 有善意的观察者见不到表面的东西.
Answer: Ｂ. 没有敏锐的观察力不可能探到人的最深处.

---

Question: 由于五代时期经常改朝换代,一切因陋就简,顾不上进行新的建设. 所以,后周以前,开封的规模同唐朝差不多. 到郭威建立后周以后,实行政治,经济改革,情况有所好转. 最能准确复述这段话主要意思的是:. 1. 代时期朝代更替频繁. 2. 战乱时期无暇顾及建设发展. 3. 代时期对于都城没有什么建设. 4. 开封的规模在五代同唐朝时差不多.
Answer: 代时期对于都城没有什么建设.

---

Question: 超过半米长的碳纳米管的问世,使其朝应用方向踏出了重要一步. 碳纳米管最重要的应用就是制造“又强又韧又轻又便宜”的材料. 在天然材料中蛛丝最强韧,而碳纳米管纤维的韧性超过了蛛丝,这样的材料在航空航天领域及其重要. 除此之外,碳纳米管还有独特的结构和导电性,可以用来做纳米尺度的电子器件,会比传统的电子器件更小,更灵敏. 但是一般的碳纳米管长度不一,结构类型和导电性不同,很难用来规模化地生产结构和性能一致的器件. 然而现在的超长碳纳米管克服了这一难题,成为一项巨大突破. 这段文字意在说明碳纳米管的? 1. 应用前景. 2. 物理特性. 3. 结构性能. 4. 研究突破.
Answer: 应用前景.

---

Question: 最近举行的一项调查表明,

In [101]:
NUM_THREADS = 32
evaluate_cot = Evaluate(devset=test_dataset_sampled, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)

In [102]:
evaluate_cot(cot_compiled)

Average Metric: 2 / 6  (33.3): 100%|██████████| 6/6 [00:01<00:00,  3.05it/s]


Average Metric: 2 / 6  (33.3%)


,question,example_answer,rationale,pred_answer,answer_exact_match
0,15. 某城市的房地产开发商只能通过向银行直接贷款或者通过预售商品房来筹集更多的开发资金。政府不允许银行增加对房地产业的直接贷款，结果使得该市的房地产开发商无法筹集到更多的开发资金，因为_______. 以下哪个选项能够合逻辑完成上述论证？. 1. 的房地产开发商预售商品房后携款潜逃，使得工程竣工遥遥无期。. 2. 央银行取消了商品房预售制度。. 3. 筑施工企业不愿意垫资施工。. 4. 分开发商销售期房后延期交房，使得很多购房者对开发商心存疑惑。.,央银行取消了商品房预售制度。.,produce the answer. We need to identify the reason why the real estate developers are unable to raise more development funds.,央银行取消了商品房预售制度。,✔️ [True]
1,40. 今年上半年，北京凯华出租汽车公司接到的乘客投诉电话是北京安达出租汽车公司的2倍，这说明安达出租汽车公司比凯华出租汽车公司的管理更规范，服务质量更高。. 如果以下陈述为真，哪一项最能支持上述结论? 1. 凯华出租汽车公司的投诉电话号码数不如安达出租汽车公司的多。. 2. 凯华出租汽车公司的投诉电话数量比安达出租汽车公司的上升得快。. 3. 安达出租汽车公司的在运营车辆是凯华出租汽车公司的2倍。. 4. 打给凯华出租汽车公司的投诉电话通常比打给安达出租汽车公司的投诉电话时间更长。.,安达出租汽车公司的在运营车辆是凯华出租汽车公司的2倍。.,produce the answer. We need to consider the factors that contribute to the number of complaints received by each company and how it reflects on...,凯华出租汽车公司的投诉电话数量比安达出租汽车公司的上升得快。,False
2,40、迄今为止，年代最久远的智人遗骸在非洲出现，距今大约20万年。据此，很多科学家认为，人类起源于非洲，现代人的直系祖先——智人在约20万年前于非洲完成进化后，然后在约15万年到20万年前，慢慢向北迁徙，穿越中东达到欧洲和亚洲，逐步迁徙至世界其他地方。. 以下哪项如果为真，最能反驳上述科学家的观点？. 1. 现代智人，生活在旧石器时代晚期，大约距今4万年至1万年左右。我国境内，许多地方都有晚期智人化石或者文化遗址发现，地点数以百计. 2. 在南美洲的一处考古发掘中，人们发现生活于大约17万年前的智人头骨化石. 3. 智人具备了个体之间能够相互沟通，能够制定计划、能够解决种种困难问题的那种非凡的能力. 4. 在以色列特拉维夫以东12公里的 Qesem 洞穴中发现了8颗40万年前的智人牙齿，这是科学家迄今为止在全球发现的年代最为久远的智人遗骸.,在以色列特拉维夫以东12公里的 Qesem 洞穴中发现了8颗40万年前的智人牙齿，这是科学家迄今为止在全球发现的年代最为久远的智人遗骸.,produce the answer. We need to identify the evidence that contradicts the theory that modern humans originated in Africa and migrated to other parts of...,在南美洲的一处考古发掘中，人们发现生活于大约17万年前的智人头骨化石.,False
3,51、某科研单位2013年新招聘的研究人员，或者具有副高以上职称的“引进人才”，或者是具有北京户籍的应届毕业的博士研究生。应届毕业的博士研究生都居住在博士后公寓中，“引进人才”都居住在“牡丹园”小区。. 关于该单位2013年新招聘的研究人员，以下哪项判断是正确的？. 1. 居住在博士后公寓的都没有副高以上职称. 2. 具有博士学位的都是具有北京户籍的. 3. 居住在“牡丹园”小区的都没有博士学位. 4. 非应届毕业的博士研究生都居住在“牡丹园”小区.,非应届毕业的博士研究生都居住在“牡丹园”小区.,produce the answer. We need to analyze the given conditions and make logical deductions based on them.,1. 居住在博士后公寓的都没有副高以上职称.,False
4,30．某大学举办围棋比赛。在进行第一轮淘汰赛后，进入第二轮的6位棋手实力相当，不过，还是可以分出高下。在已经进行的两轮比赛中，棋手甲战胜了棋手乙，棋手乙掌声了棋手丙。明天，棋手甲和丙将进行比赛。. 请根据题干，从逻辑上预测比赛结果：. 1. 手甲肯定会赢. 2. 手丙肯定会赢. 3. 人将战成平局. 4. 手甲很可能赢，但也有可能输.,手甲很可能赢，但也有可能输.,"produce the answer. We know that player A defeated player B, and player B defeated player C. Therefore, player A has a higher chance of...",手甲很可能赢，但也有可能输.,✔️ [True]
5,某一公司有一栋6层的办公楼，公司的财务部、企划部、行政部、销售部、人力资源部、研发部等6个部门在此办公，每个部门占据其中的一层。已知：（1）人力资源部、销售部两个部门所在的楼层不相邻；（2）财务部在企划部下一层；（3）行政部所在的楼层在企划部的上面，但是在人力资源部的下面。. 46、以下哪项可能分别是第一层、第二层所在的两个部门？. 1. 财务部、销售部. 2. 企划部、销售部. 3. 研发部、销售部. 4. 销售部、企划部.,研发部、销售部.,produce the answer. We need to consider the given conditions and determine the possible arrangement of departments on each floor.,销售部、企划部.,False


33.33